# AML Assignment-2

| Name                | ID         |
|---------------------|------------|
| Ananya Sinha        | MDS202307  |
| Divyanshi Kumari    | MDS202322  |
| Rohit Roy           | MDS202340  |

# Question 2

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals 
import numpy as np 
import tensorflow as tf 
from keras.models import Sequential 
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, SimpleRNN
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint 
from keras.callbacks import ReduceLROnPlateau 
import random 
import pandas as pd

2024-10-27 12:38:03.509915: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-27 12:38:03.524992: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-27 12:38:03.529563: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-27 12:38:03.540736: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-27 12:38:04.271066: W tensorflow/compiler/tf2

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### Converting the SMS dataframe to a txt file with end character

In [ ]:
sms_df = pd.read_csv('data/spam.csv', encoding='ISO-8859-1')
sms_df.dropna(how="any", inplace=True, axis=1)
sms_df.columns = ['label', 'message']
sms_df.head()

In [ ]:
# Define the end character you want to add
end_character = '¤'

# Save messages to a text file with the end character
with open('data/sms_messages.txt', 'w') as f:
    for message in sms_df['message']:
        f.write(f"{message} {end_character}\n")

### Preparing data for training

In [ ]:
# Reading the text file into a string 
with open('data/sms_messages.txt', 'r') as file: 
    text = file.read() 

In [4]:
# Storing all the unique characters present in the text 
vocabulary = sorted(list(set(text))) 
  
# Creating dictionaries to map each character to an index 
char_to_indices = dict((c, i) for i, c in enumerate(vocabulary)) 
indices_to_char = dict((i, c) for i, c in enumerate(vocabulary)) 
  
print(vocabulary) 

['\n', ' ', '!', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '~', '\x89', '\x8b', '\x8e', '£', '¤', '©', 'ª', '¬', '´', '¼', 'Á', 'Â', 'È', 'Ì', 'Ï', 'Ð', 'Ò', 'Ó', 'Ô', 'Õ', 'Û', 'ä', 'å', 'ö', '÷']


In [9]:
max_length = 100
steps = 5
sentences = [] 
next_chars = [] 
for i in range(0, len(text) - max_length, steps): 
    sentences.append(text[i: i + max_length]) 
    next_chars.append(text[i + max_length]) 
      
# Hot encoding each character into a boolean vector 
X = np.zeros((len(sentences), max_length, len(vocabulary)), dtype = bool) 
y = np.zeros((len(sentences), len(vocabulary)), dtype = bool) 
for i, sentence in enumerate(sentences): 
    for t, char in enumerate(sentence): 
        X[i, t, char_to_indices[char]] = 1
    y[i, char_to_indices[next_chars[i]]] = 1

### Defining the models

In [10]:
# RNN
model_rnn = Sequential() 
model_rnn.add(SimpleRNN(278, input_shape =(max_length, len(vocabulary))))
model_rnn.add(Dropout(0.2))
model_rnn.add(Dense(len(vocabulary))) 
model_rnn.add(Activation('softmax')) 
optimizer = RMSprop(learning_rate= 0.01) 
model_rnn.compile(loss ='categorical_crossentropy', optimizer = optimizer) 
model_rnn.summary()

# LSTM
model_lstm = Sequential() 
model_lstm.add(LSTM(128, input_shape =(max_length, len(vocabulary)))) 
model_lstm.add(Dense(len(vocabulary))) 
model_lstm.add(Activation('softmax')) 
optimizer = RMSprop(learning_rate= 0.01) 
model_lstm.compile(loss ='categorical_crossentropy', optimizer = optimizer) 
model_lstm.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_1 (SimpleRNN)        │ (None, 278)            │       110,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 278)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 117)            │        32,643 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 117)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 142,731 (557.54 KB)

 Trainable params: 142,731 (557.54 KB)

 Non-trainable params: 0 (0.00 B)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 128)            │       125,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 117)            │        15,093 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 117)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 141,045 (550.96 KB)

 Trainable params: 141,045 (550.96 KB)

 Non-trainable params: 0 (0.00 B)

### Model Parameters :

RNN : 142,731

LSTM : 141,045

### Defining callbacks

In [11]:
# Defining a helper function to save the model after each epoch in which the loss decreases 
filepath_rnn = "weights_rnn_2.keras"
checkpoint_rnn = ModelCheckpoint(filepath_rnn, monitor ='loss', 
                             verbose = 1, save_best_only = True, 
                             mode ='min')

# Defining a helper function to reduce the learning rate each time the learning plateaus 
reduce_alpha_rnn = ReduceLROnPlateau(monitor ='loss', factor = 0.2, 
                              patience = 1, min_lr = 0.001) 
callbacks_rnn = [checkpoint_rnn, reduce_alpha_rnn] 

In [21]:
# Defining a helper function to save the model after each epoch in which the loss decreases 
filepath_lstm = "weights_lstm.keras"
checkpoint_lstm = ModelCheckpoint(filepath_lstm, monitor ='loss', 
                             verbose = 1, save_best_only = True, 
                             mode ='min')

# Defining a helper function to reduce the learning rate each time the learning plateaus 
reduce_alpha_lstm = ReduceLROnPlateau(monitor ='loss', factor = 0.2, 
                              patience = 1, min_lr = 0.001) 
callbacks_lstm = [checkpoint_lstm, reduce_alpha_lstm] 

### Training the models

In [12]:
# Training the RNN model 
model_rnn.fit(X, y, batch_size = 128, epochs = 30, callbacks = callbacks_rnn)

Epoch 1/30
723/724 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 4.8622
Epoch 1: loss improved from inf to 4.54480, saving model to weights_rnn_2.keras
724/724 ━━━━━━━━━━━━━━━━━━━━ 35s 48ms/step - loss: 4.8614 - learning_rate: 0.0100
Epoch 2/30
723/724 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 4.2839
Epoch 2: loss improved from 4.54480 to 4.25632, saving model to weights_rnn_2.keras
724/724 ━━━━━━━━━━━━━━━━━━━━ 35s 48ms/step - loss: 4.2838 - learning_rate: 0.0100
Epoch 3/30
723/724 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 4.2934
Epoch 3: loss did not improve from 4.25632
724/724 ━━━━━━━━━━━━━━━━━━━━ 35s 48ms/step - loss: 4.2933 - learning_rate: 0.0100
Epoch 4/30
724/724 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 3.6083
Epoch 4: loss improved from 4.25632 to 3.55393, saving model to weights_rnn_2.keras
724/724 ━━━━━━━━━━━━━━━━━━━━ 35s 48ms/step - loss: 3.6082 - learning_rate: 0.0020
Epoch 5/30
723/724 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 3.5238
Epoch 5: loss improved from 3.55393 to 3.

In [22]:
# Training the LSTM model 
model_lstm.fit(X, y, batch_size = 128, epochs = 30, callbacks = callbacks_lstm)

Epoch 1/30
723/724 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: 2.9723
Epoch 1: loss improved from inf to 2.62739, saving model to weights_lstm.keras
724/724 ━━━━━━━━━━━━━━━━━━━━ 68s 93ms/step - loss: 2.9713 - learning_rate: 0.0100
Epoch 2/30
723/724 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - loss: 2.2141
Epoch 2: loss improved from 2.62739 to 2.16912, saving model to weights_lstm.keras
724/724 ━━━━━━━━━━━━━━━━━━━━ 67s 92ms/step - loss: 2.2140 - learning_rate: 0.0100
Epoch 3/30
723/724 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: 2.0061
Epoch 3: loss improved from 2.16912 to 2.00081, saving model to weights_lstm.keras
724/724 ━━━━━━━━━━━━━━━━━━━━ 67s 93ms/step - loss: 2.0061 - learning_rate: 0.0100
Epoch 4/30
723/724 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: 1.8848
Epoch 4: loss improved from 2.00081 to 1.88887, saving model to weights_lstm.keras
724/724 ━━━━━━━━━━━━━━━━━━━━ 67s 93ms/step - loss: 1.8848 - learning_rate: 0.0100
Epoch 5/30
723/724 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: 1.7856
Epoc

In [23]:
import joblib
joblib.dump(model_rnn, 'rnn_final.pickle')
joblib.dump(model_lstm, 'lstm_final.pickle')

['lstm_final.pickle']

### Sample Text Generation

In [13]:
# Helper function to sample an index from a probability array 
def sample_index(preds, temperature = 1.0): 
    preds = np.asarray(preds).astype('float64') 
    preds = np.log(preds) / temperature 
    exp_preds = np.exp(preds) 
    preds = exp_preds / np.sum(exp_preds) 
    probas = np.random.multinomial(1, preds, 1) 
    return np.argmax(probas) 

In [24]:
def generate_text(length, max_length, model, diversity): 
    # Get random starting text 
    start_index = random.randint(0, len(text) - max_length - 1) 
    generated = '' 
    sentence = text[start_index: start_index + max_length]
    # sentence = 'sey Devils and the Detroit Red Wings play Ice'
    print("----Seed SMS----")
    print(sentence)
    generated += sentence 
    next_char = ''
    for _ in range(length):
        x_pred = np.zeros((1, max_length, len(vocabulary))) 
        for t, char in enumerate(sentence): 
            x_pred[0, t, char_to_indices[char]] = 1.

        preds = model.predict(x_pred, verbose = 0)[0] 
        next_index = sample_index(preds, diversity)
        next_char = indices_to_char[next_index] 

        generated += next_char 
        sentence = sentence[1:] + next_char 
        if next_char == '¤':
                break
    print("----Generated SMS----")
    print(generated) 

#### Generation by RNN

In [26]:
# Sample generation by RNN model 
generate_text(200, max_length, model_rnn, 0.2)

----Seed SMS----
lunch on my way home lor... I tot u dun wan 2 stay in sch today... ¤
K then 2marrow are you coming t
----Generated SMS----
lunch on my way home lor... I tot u dun wan 2 stay in sch today... ¤
K then 2marrow are you coming tat and an lore the you alle you low you all you and and in the ane you the to you lore tall an ¤


#### Generation by LSTM

In [25]:
# Sample generation by LSTM model
generate_text(200, max_length, model_lstm, 0.2)

----Seed SMS----
puts things into perspective when something like this happens ¤
Now got tv 2 watch meh? U no work to
----Generated SMS----
puts things into perspective when something like this happens ¤
Now got tv 2 watch meh? U no work to drive or something of the we was money and still but if I can still be watch my start so i am late. ¤
